In [10]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [32]:
n_in, n_h, n_out, batch_size = 3, 500, 1, 50

In [33]:
data_raw = pd.read_csv('saved.csv')
data_raw.columns = ['vid', 'init_time', 'tls_distance', 'stop_distance', 'time_to_green',
                'tls_light_count', 'time_of_pass', 'leading_cars', 'stop_leading_cars', 'stop_duration']

## Data validation

In [34]:
data_down = data_raw[data_raw.vid.str.startswith('down')]
data_left = data_raw[data_raw.vid.str.startswith('left')]
data_right = data_raw[data_raw.vid.str.startswith('right')]

In [35]:
data_raw[data_raw.leading_cars < data_raw.stop_leading_cars]

,vid,init_time,tls_distance,stop_distance,time_to_green,tls_light_count,time_of_pass,leading_cars,stop_leading_cars,stop_duration


In [39]:
data_raw.head()

,vid,init_time,tls_distance,stop_distance,time_to_green,tls_light_count,time_of_pass,leading_cars,stop_leading_cars,stop_duration
0,down_1,56.0,1.001033,1.001033,34.0,1,97.0,0,0,41.0
1,down_2,56.0,8.502086,8.502086,34.0,1,102.0,1,1,42.0
2,down_6,56.0,16.011007,16.003093,34.0,1,105.0,2,2,37.0
3,down_7,56.0,71.341208,23.504144,34.0,1,108.0,3,3,34.0
4,down_20,56.0,389.348443,53.508408,34.0,1,118.0,7,7,1.0


In [54]:
data_down.to_csv('down.csv', index=False)

In [55]:
data_left.to_csv('left.csv', index=False)

In [56]:
data_right.to_csv('right.csv', index=False)

In [57]:
data_left.sort_values('init_time')

,vid,init_time,tls_distance,stop_distance,time_to_green,tls_light_count,time_of_pass,leading_cars,stop_leading_cars,stop_duration
7,left_26,56.0,499.702099,1.001045,0.0,0,193.0,7,0,86.0
10,left_34,111.0,35.632039,23.504181,75.0,1,204.0,3,3,77.0
12,left_45,111.0,317.213684,46.007346,75.0,1,212.0,6,6,64.0
13,left_53,111.0,411.205189,53.508364,75.0,1,214.0,7,7,59.0
19,left_165,276.0,148.093397,1.001002,0.0,0,385.0,1,0,93.0
21,left_169,276.0,267.487041,8.502110,0.0,0,390.0,2,1,7.0
23,left_199,331.0,97.453917,68.510302,47.0,1,410.0,9,9,53.0
27,left_221,331.0,472.208358,98.514505,47.0,1,418.0,13,13,32.0
29,left_247,386.0,384.573642,166.023895,0.0,0,435.0,21,14,1.0
31,left_272,441.0,78.524297,-1.000000,0.0,0,449.0,3,-1,-1.0


## Learning

In [45]:
data = data_raw[['tls_distance', 'time_to_green', 'leading_cars', 'stop_duration']]

In [46]:
x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)
data.columns = ['tls_distance', 'time_to_green', 'leading_cars', 'stop_duration']

In [47]:
data.head()

,tls_distance,time_to_green,leading_cars,stop_duration
0,0.001790,0.357895,0.000000,0.428571
1,0.016790,0.357895,0.037037,0.438776
2,0.031805,0.357895,0.074074,0.387755
3,0.142444,0.357895,0.111111,0.357143
4,0.778338,0.357895,0.259259,0.020408


In [48]:
# split data to training and testing
train_set = data.sample(frac=0.75, random_state=0)
test_set = data.drop(train_set.index)
train_set_labels = train_set[['stop_duration']]
test_set_labels = test_set_labels = test_set[['stop_duration']]
train_set = train_set[['tls_distance', 'time_to_green', 'leading_cars']]
test_set = test_set[['tls_distance', 'time_to_green', 'leading_cars']]

In [49]:
X = torch.tensor(train_set.values, dtype=torch.float32)
y = torch.tensor(train_set_labels.values, dtype=torch.float32)
X_test = torch.tensor(test_set.values, dtype=torch.float32)
y_test = torch.tensor(test_set_labels.values, dtype=torch.float32)

In [50]:
model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out),
                     nn.Sigmoid())

In [51]:
criterion = torch.nn.MSELoss()

In [52]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [53]:
for epoch in range(50000):
    # Forward Propagation
    y_pred = model(X)    # Compute and print loss
    loss = criterion(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())    # Zero the gradients
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

epoch:  0  loss:  0.15741291642189026
epoch:  1  loss:  0.15611104667186737
epoch:  2  loss:  0.1549176573753357
epoch:  3  loss:  0.15382245182991028
epoch:  4  loss:  0.152816042304039
epoch:  5  loss:  0.15188980102539062
epoch:  6  loss:  0.15103591978549957
epoch:  7  loss:  0.150247260928154
epoch:  8  loss:  0.14951740205287933
epoch:  9  loss:  0.14884053170681
epoch:  10  loss:  0.14821140468120575
epoch:  11  loss:  0.14762528240680695
epoch:  12  loss:  0.14707790315151215
epoch:  13  loss:  0.14656546711921692
epoch:  14  loss:  0.1460845172405243
epoch:  15  loss:  0.14563198387622833
epoch:  16  loss:  0.1452050805091858
epoch:  17  loss:  0.1448013335466385
epoch:  18  loss:  0.14441850781440735
epoch:  19  loss:  0.1440546065568924
epoch:  20  loss:  0.14370784163475037
epoch:  21  loss:  0.14337657392024994
epoch:  22  loss:  0.1430594027042389
epoch:  23  loss:  0.14275498688220978
epoch:  24  loss:  0.14246217906475067
epoch:  25  loss:  0.1421799510717392
epoch:  26

epoch:  214  loss:  0.11987341195344925
epoch:  215  loss:  0.1198112890124321
epoch:  216  loss:  0.11974947154521942
epoch:  217  loss:  0.11968794465065002
epoch:  218  loss:  0.1196267157793045
epoch:  219  loss:  0.11956577748060226
epoch:  220  loss:  0.1195051372051239
epoch:  221  loss:  0.11944478005170822
epoch:  222  loss:  0.11938471347093582
epoch:  223  loss:  0.1193249300122261
epoch:  224  loss:  0.11926542222499847
epoch:  225  loss:  0.11920620501041412
epoch:  226  loss:  0.11914725601673126
epoch:  227  loss:  0.11908858269453049
epoch:  228  loss:  0.1190301850438118
epoch:  229  loss:  0.1189720556139946
epoch:  230  loss:  0.11891419440507889
epoch:  231  loss:  0.11885660886764526
epoch:  232  loss:  0.11879927664995193
epoch:  233  loss:  0.1187422126531601
epoch:  234  loss:  0.11868540942668915
epoch:  235  loss:  0.1186288595199585
epoch:  236  loss:  0.11857256293296814
epoch:  237  loss:  0.11851652711629868
epoch:  238  loss:  0.1184607446193695
epoch:  2

epoch:  423  loss:  0.11102231591939926
epoch:  424  loss:  0.1109924465417862
epoch:  425  loss:  0.1109626516699791
epoch:  426  loss:  0.11093293130397797
epoch:  427  loss:  0.1109032854437828
epoch:  428  loss:  0.11087371408939362
epoch:  429  loss:  0.1108442097902298
epoch:  430  loss:  0.11081477999687195
epoch:  431  loss:  0.11078542470932007
epoch:  432  loss:  0.11075613647699356
epoch:  433  loss:  0.11072692275047302
epoch:  434  loss:  0.11069778352975845
epoch:  435  loss:  0.11066871136426926
epoch:  436  loss:  0.11063970625400543
epoch:  437  loss:  0.11061077564954758
epoch:  438  loss:  0.1105819121003151
epoch:  439  loss:  0.11055312305688858
epoch:  440  loss:  0.11052440106868744
epoch:  441  loss:  0.11049574613571167
epoch:  442  loss:  0.11046715825796127
epoch:  443  loss:  0.11043864488601685
epoch:  444  loss:  0.1104101911187172
epoch:  445  loss:  0.11038181185722351
epoch:  446  loss:  0.1103534922003746
epoch:  447  loss:  0.11032524704933167
epoch: 

epoch:  633  loss:  0.10597707331180573
epoch:  634  loss:  0.10595756024122238
epoch:  635  loss:  0.10593807697296143
epoch:  636  loss:  0.10591863095760345
epoch:  637  loss:  0.10589922219514847
epoch:  638  loss:  0.10587983578443527
epoch:  639  loss:  0.10586049407720566
epoch:  640  loss:  0.10584117472171783
epoch:  641  loss:  0.105821892619133
epoch:  642  loss:  0.10580264776945114
epoch:  643  loss:  0.10578343272209167
epoch:  644  loss:  0.1057642474770546
epoch:  645  loss:  0.1057450920343399
epoch:  646  loss:  0.1057259738445282
epoch:  647  loss:  0.10570689290761948
epoch:  648  loss:  0.10568783432245255
epoch:  649  loss:  0.1056688129901886
epoch:  650  loss:  0.10564982146024704
epoch:  651  loss:  0.10563086718320847
epoch:  652  loss:  0.10561194270849228
epoch:  653  loss:  0.10559304803609848
epoch:  654  loss:  0.10557418316602707
epoch:  655  loss:  0.10555534809827805
epoch:  656  loss:  0.105536550283432
epoch:  657  loss:  0.10551778227090836
epoch:  

epoch:  843  loss:  0.1024959534406662
epoch:  844  loss:  0.1024818941950798
epoch:  845  loss:  0.102467842400074
epoch:  846  loss:  0.1024538204073906
epoch:  847  loss:  0.10243981331586838
epoch:  848  loss:  0.10242582857608795
epoch:  849  loss:  0.10241186618804932
epoch:  850  loss:  0.10239791870117188
epoch:  851  loss:  0.10238399356603622
epoch:  852  loss:  0.10237008333206177
epoch:  853  loss:  0.1023562029004097
epoch:  854  loss:  0.10234233736991882
epoch:  855  loss:  0.10232848674058914
epoch:  856  loss:  0.10231466591358185
epoch:  857  loss:  0.10230085253715515
epoch:  858  loss:  0.10228706896305084
epoch:  859  loss:  0.10227330029010773
epoch:  860  loss:  0.1022595539689064
epoch:  861  loss:  0.10224582254886627
epoch:  862  loss:  0.10223211348056793
epoch:  863  loss:  0.10221842676401138
epoch:  864  loss:  0.10220475494861603
epoch:  865  loss:  0.10219110548496246
epoch:  866  loss:  0.10217747837305069
epoch:  867  loss:  0.10216386616230011
epoch: 

KeyboardInterrupt: 

In [26]:
y_pred = model(X_test)
dif = (y_pred - y_test)**2

In [27]:
MSE = dif.mean()
MSE

tensor(0.0927, grad_fn=<MeanBackward0>)